In [3]:
pip install pandas sqlalchemy pymysql

In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import json

In [2]:
import zipfile

zip_path = "mixed_dataset.zip"
extract_path = "mixed_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted to:", extract_path)

Files extracted to: mixed_dataset


In [9]:
from sqlalchemy import create_engine, text
import urllib.parse

username = "noname"
password = urllib.parse.quote_plus("noname")
host = "localhost"
database = "noname"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
print("Connected to MySQL successfully.")

Connected to MySQL successfully.


In [19]:
csv_folder = os.path.join(extract_path, "csv")

for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        path = os.path.join(csv_folder, file)
        table_name = os.path.splitext(file)[0]
        
        df = pd.read_csv(path)
        if not df.empty:
            df.to_sql(table_name, con=engine, index=False, if_exists='append')
            with engine.connect() as conn:
                count = conn.execute(text(f"SELECT COUNT(*) FROM `{table_name}`")).scalar()
            print(f"{file} → {table_name}: {len(df)} rows added (Total now: {count})")
        else:
            print(f"{file} is empty, skipped.")

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'name' in 'field list'")
[SQL: INSERT INTO data_0 (id, name, email, age, balance, is_active, signup_date, last_login) VALUES (%(id)s, %(name)s, %(email)s, %(age)s, %(balance)s, %(is_active)s, %(signup_date)s, %(last_login)s)]
[parameters: [{'id': 1, 'name': 'Amanda Aguilar', 'email': 'edward07@green.biz', 'age': 60.0, 'balance': 9126.82, 'is_active': 1, 'signup_date': '2021-11-04', 'last_login': None}, {'id': 2, 'name': 'Gregory Washington', 'email': None, 'age': 18.0, 'balance': 8899.89, 'is_active': 1, 'signup_date': '2025-09-11', 'last_login': None}, {'id': 3, 'name': 'Daniel Stephens', 'email': 'smithjennifer@hotmail.com', 'age': 42.0, 'balance': 9575.99, 'is_active': 1, 'signup_date': '2020-10-18', 'last_login': '2025-08-10 02:26:58'}, {'id': 4, 'name': 'Joshua Greene', 'email': 'jeremy90@gmail.com', 'age': 41.0, 'balance': 2516.23, 'is_active': 0, 'signup_date': '2020-04-13', 'last_login': None}, {'id': 5, 'n

In [12]:
json_folder = os.path.join(extract_path, "json")

for file in os.listdir(json_folder):
    if file.endswith(".json"):
        path = os.path.join(json_folder, file)
        table_name = os.path.splitext(file)[0]

        with open(path, 'r') as f:
            data = json.load(f)
        df = pd.json_normalize(data)

        # Convert nested fields
        for col in df.columns:
            df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

        if df.empty:
            print(f"{file} is empty, skipped.")
            continue

        # Try appending; recreate on schema mismatch
        try:
            df.to_sql(table_name, con=engine, index=False, if_exists='append')
        except Exception as e:
            print(f" Schema mismatch or column error in {table_name}: {e}")
            df.to_sql(table_name, con=engine, index=False, if_exists='replace')
            print(f"{file} → {table_name} recreated successfully.")

        with engine.connect() as conn:
            count = conn.execute(text(f"SELECT COUNT(*) FROM `{table_name}`")).scalar()
        print(f"{file} → {table_name}: {len(df)} rows added (Total now: {count})")


⚠️ Schema mismatch or column error in data_0: (pymysql.err.OperationalError) (1054, "Unknown column 'created_at' in 'field list'")
[SQL: INSERT INTO data_0 (id, created_at, `user.name`, `user.email`, `user.tags`, `metrics.score`, `metrics.level`, `metrics.active`, `metadata.notes`, metadata) VALUES (%(id)s, %(created_at)s, %(user_name)s, %(user_email)s, %(user_tags)s, %(metrics_score)s, %(metrics_level)s, %(metrics_active)s, %(metadata_notes)s, %(metadata)s)]
[parameters: [{'id': 309901, 'created_at': '2025-05-28 18:56:54', 'user_name': 'Ryan Fox', 'user_email': 'christine29@patterson.info', 'user_tags': '["want", "over", "write"]', 'metrics_score': 72.41, 'metrics_level': 7, 'metrics_active': 1, 'metadata_notes': 'Window after right notice.', 'metadata': None}, {'id': 328229, 'created_at': '2025-06-12 06:14:02', 'user_name': 'Stacey Porter', 'user_email': 'hthompson@cruz-clark.net', 'user_tags': '["hope", "relate", "grow", "administration", "free"]', 'metrics_score': 63.83, 'metrics_l

In [13]:
text_folder = os.path.join(extract_path, "text")

# Ensure table exists
with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS text_files (
            id INT AUTO_INCREMENT PRIMARY KEY,
            file_name VARCHAR(255),
            content LONGTEXT
        )
    """))


with engine.begin() as conn:
    for file in os.listdir(text_folder):
        if file.endswith((".txt", ".log")):
            path = os.path.join(text_folder, file)
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()

            chunk_size = 50000
            inserted_rows = 0
            for i in range(0, len(content), chunk_size):
                chunk = content[i:i + chunk_size]
                stmt = text("INSERT INTO text_files (file_name, content) VALUES (:file_name, :content)")
                conn.execute(stmt, {"file_name": file, "content": chunk})
                inserted_rows += 1

            print(f"{file} → text_files: {inserted_rows} chunks inserted.")


✅ data_0.txt → text_files: 105 chunks inserted.
✅ data_1.txt → text_files: 105 chunks inserted.


In [14]:
urls_folder = os.path.join(extract_path, "urls")


with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS urls (
            id INT AUTO_INCREMENT PRIMARY KEY,
            file_name VARCHAR(255),
            url TEXT
        )
    """))

for file in os.listdir(urls_folder):
    path = os.path.join(urls_folder, file)
    with open(path, 'r', encoding='utf-8') as f:
        lines = [l.strip() for l in f if l.strip()]

    if lines:
        df = pd.DataFrame({"file_name": [file]*len(lines), "url": lines})
        df.to_sql("urls", con=engine, index=False, if_exists='append')

        with engine.connect() as conn:
            count = conn.execute(text("SELECT COUNT(*) FROM urls")).scalar()
        print(f"{file} → urls: {len(lines)} rows added (Total now: {count})")
    else:
        print(f" {file} is empty, skipped.")


✅ data_0.txt → urls: 5000 rows added (Total now: 5000)


In [16]:
tables = pd.read_sql("SHOW TABLES;", engine)
print("📋 Tables in database:")
display(tables)


📋 Tables in database:


,Tables_in_randommigration
0,data_0
1,data_1
2,text_files
3,urls


In [17]:
with engine.connect() as conn:
    for (table,) in conn.execute(text("SHOW TABLES")).fetchall():
        count = conn.execute(text(f"SELECT COUNT(*) FROM `{table}`")).scalar()
        print(f"{table}: {count} rows")


data_0: 5000 rows
data_1: 5000 rows
text_files: 210 rows
urls: 5000 rows
